In [9]:
# Install if needed
#!pip install rdflib
#!pip install networkx

In [10]:
# Import von rdflib, networkx und andere libraries
from rdflib import Graph, Namespace, RDF
import networkx as nx
from collections import deque
import matplotlib.pyplot as plt

In [11]:
# === 4. BFS with class filtering ===
def bfs_with_class_filter(G, start_node, target_class_uri):
    visited = set()
    queue = deque([start_node])
    matched_nodes = []

    while queue:
        node = queue.popleft()
        if node in visited:
            continue
        visited.add(node)

        if G.nodes[node].get('class') == target_class_uri:
            matched_nodes.append(node)

        for neighbor in G.neighbors(node):
            if neighbor not in visited:
                queue.append(neighbor)

    return matched_nodes

In [13]:
# Gerichtete Graph erstellen
def graph_from_rdf(g):
    
    G = nx.DiGraph()
    
    # Extract rdf:type information
    types = {}
    for s, p, o in g:
        if p == RDF.type:
            types[str(s)] = str(o)
    
    # Add all triples except rdf:type as edges
    for s, p, o in g:
        s_str, o_str, p_str = str(s), str(o), str(p)
        G.add_edge(s_str, o_str, label=p_str)
    
    # Add node classes as attributes
    for node in G.nodes:
        G.nodes[node]['class'] = types.get(node, None)

    # === 3. Show the entire KG ===
    print("Knowledge Graph:")
    for u, v, d in G.edges(data=True):
        label = d['label'].split("/")[-1]
        print(f"{u.split('/')[-1]} --[{label}]--> {v.split('/')[-1]}")

    
    return G

In [12]:
g = Graph()

# Namespace definieren
SC = Namespace("http://example.org/")

# Triples
g.add((SC.supplyChain1, RDF.type, SC.SupplyChain))
g.add((SC.teilzulieferer, RDF.type, SC.Firma))
g.add((SC.hersteller, RDF.type, SC.Firma))
g.add((SC.valueStreamHersteller, RDF.type, SC.ValueStream))

g.add((SC.hersteller, SC.istTeilVon, SC.supplyChain1))
g.add((SC.valueStreamHersteller, SC.istTeilVon, SC.hersteller))

<Graph identifier=N1a076c7967474e7caae2ed6b925442d0 (<class 'rdflib.graph.Graph'>)>

In [14]:
G = graph_from_rdf(g)

Knowledge Graph:
supplyChain1 --[22-rdf-syntax-ns#type]--> SupplyChain
hersteller --[22-rdf-syntax-ns#type]--> Firma
hersteller --[istTeilVon]--> supplyChain1
valueStreamHersteller --[istTeilVon]--> hersteller
valueStreamHersteller --[22-rdf-syntax-ns#type]--> ValueStream
teilzulieferer --[22-rdf-syntax-ns#type]--> Firma


In [15]:
start_node = str(SC.supplyChain1)
target_class = str(SC.ValueStream)

# Rückwarts traversal BFS
rwarts_G = G.reverse()

results = bfs_with_class_filter(rwarts_G, start_node, target_class)

print("BFS Result (full URIs):", results)

BFS Result (full URIs): ['http://example.org/valueStreamHersteller']
